In [5]:

import os
import json
import snowflake.connector
import requests
from dotenv import load_dotenv
from smolagents import CodeAgent, tool
from smolagents.agents import ActionStep
from smolagents import LiteLLMModel

import pandas as pd
from typing import Dict, Any, List, Optional
from tavily import TavilyClient

# Load environment variables
load_dotenv()

True

In [6]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

## Helper functions

In [7]:
def get_snowflake_connection():
    """Creates and returns a Snowflake connection."""
    return snowflake.connector.connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        database=os.getenv("SNOWFLAKE_DATABASE"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        schema=os.getenv("SNOWFLAKE_SCHEMA")
    )

def close_connection(cursor, conn):
    """Closes a Snowflake cursor and connection."""
    if cursor:
        cursor.close()
    if conn:
        conn.close()

### Snowflake tools

In [8]:
@tool
def query_covid_cases_by_year(state: Optional[str] = None) -> str:
    """
    Retrieves year-over-year COVID-19 cases and deaths data.
    
    Args:
        state: Optional state filter. If None, returns data for all states.
    
    Returns:
        JSON string with year-over-year COVID cases and deaths.
    """
    try:
        conn = get_snowflake_connection()
        cursor = conn.cursor()
        
        # Build the query with optional state filter
        base_query = """
        WITH state_agg AS 
        (
            SELECT 
                EXTRACT(YEAR FROM date) AS year, 
                state, 
                MAX(cases) AS cases, 
                MAX(deaths) AS deaths 
            FROM COVID19_GLOBAL_DATA_ATLAS.HLS_COVID19_USA.COVID19_USA_CASES_DEATHS_BY_STATE_DAILY_NYT
        """
        
        if state:
            base_query += f"    WHERE state='{state}'\n"
        
        query = base_query + """
            GROUP BY EXTRACT(YEAR FROM date), state
            ORDER BY state, year)
        SELECT year, state,
            cases - LAG(cases, 1, 0) OVER (PARTITION BY state ORDER BY state, year) as cases,
            deaths - LAG(deaths, 1, 0) OVER (PARTITION BY state ORDER BY state, year) as deaths    
        FROM state_agg;
        """
        
        cursor.execute(query)
        results = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(results, columns=column_names)
        
        close_connection(cursor, conn)
        return df.to_json(orient="records")
    
    except Exception as e:
        return f"Error executing COVID cases query: {str(e)}"

@tool
def query_vaccine_providers(state: Optional[str] = None) -> str:
    """
    Retrieves data about COVID-19 vaccination provider locations.
    
    Args:
        state: Optional state filter. If None, returns data for all states.
    
    Returns:
        JSON string with vaccination provider counts by state.
    """
    us_states = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}
    try:
        conn = get_snowflake_connection()
        cursor = conn.cursor()
        
        query = """
        SELECT LOC_ADMIN_STATE, COUNT(*) AS PROVIDER_COUNT 
        FROM COVID19_GLOBAL_DATA_ATLAS.HLS_COVID19_USA.COVID_19_US_VACCINATING_PROVIDER_LOCATIONS
        """
        
        if state:
            state = us_states[state]
            print("State is : --> ", state)
            query += f" WHERE LOC_ADMIN_STATE = '{state}'"
        
        query += " GROUP BY LOC_ADMIN_STATE ORDER BY LOC_ADMIN_STATE"
        
        cursor.execute(query)
        results = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(results, columns=column_names)
        
        close_connection(cursor, conn)
        return df.to_json(orient="records")
    
    except Exception as e:
        return f"Error executing vaccine providers query: {str(e)}"

@tool
def query_healthcare_access(state: Optional[str] = None) -> str:
    """
    Retrieves healthcare access data including emergency departments, physician visits, 
    and delayed healthcare due to cost.
    
    Args:
        state: Optional state filter. If None, returns national data.
    
    Returns:
        JSON string with healthcare access data.
    """
    try:
        conn = get_snowflake_connection()
        cursor = conn.cursor()
        
        # Set the database
        # cursor.execute("USE DATABASE DIVERSITY_EQUITY_AND_INCLUSION__ACCESS_TO_HEALTHCARE")
        
        # Dictionary to store results
        results = {
            "emergency_dept_visits": [],
            "physician_visits": [],
            "delayed_healthcare_by_year": []
        }
        
        # Query 1: Emergency department visits
        query1 = """
        SELECT PANEL, UNIT, YEAR, ESTIMATE 
        FROM DIVERSITY_EQUITY_AND_INCLUSION__ACCESS_TO_HEALTHCARE.DEI_HEALTHCARE."Healthcare Visits by Age/Sex/Race - USA"
        WHERE AGE = 'All ages' AND ESTIMATE > 0 AND UNIT = 'Number of visits in thousands'
        AND PANEL = 'Hospital emergency departments'
        """
        
        # if state:
        #     query1 += f" AND STATE = '{state}'"
            
        query1 += " ORDER BY YEAR DESC"
        
        cursor.execute(query1)
        results1 = cursor.fetchall()
        column_names1 = [desc[0] for desc in cursor.description]
        df1 = pd.DataFrame(results1, columns=column_names1)
        results["emergency_dept_visits"] = json.loads(df1.to_json(orient="records"))
        
        # Query 2: Physician office visits
        query2 = """
        SELECT PANEL, UNIT, YEAR, ESTIMATE 
        FROM DIVERSITY_EQUITY_AND_INCLUSION__ACCESS_TO_HEALTHCARE.DEI_HEALTHCARE."Healthcare Visits by Age/Sex/Race - USA"
        WHERE AGE = 'All ages' AND ESTIMATE > 0 AND UNIT = 'Number of visits in thousands'
        AND PANEL = 'Physician offices'
        """
        
        # if state:
        #     query2 += f" AND STATE = '{state}'"
            
        query2 += " ORDER BY YEAR DESC"
        
        cursor.execute(query2)
        results2 = cursor.fetchall()
        column_names2 = [desc[0] for desc in cursor.description]
        df2 = pd.DataFrame(results2, columns=column_names2)
        results["physician_visits"] = json.loads(df2.to_json(orient="records"))
        
        # # Query 3: Delayed healthcare due to cost
        # query3 = "SELECT * FROM DIVERSITY_EQUITY_AND_INCLUSION__ACCESS_TO_HEALTHCARE.DEI_HEALTHCARE.\"Delayed Healthcare Due to Cost - USA\""
        
        # # if state:
        # #     query3 += f" WHERE STATE = '{state}'"
        
        # cursor.execute(query3)
        # results3 = cursor.fetchall()
        # column_names3 = [desc[0] for desc in cursor.description]
        # df3 = pd.DataFrame(results3, columns=column_names3)
        # results["delayed_healthcare"] = json.loads(df3.to_json(orient="records"))
        
        # Query 4: Delayed healthcare by year
        query4 = """
        SELECT YEAR, COUNT(*) AS COUNT 
        FROM DIVERSITY_EQUITY_AND_INCLUSION__ACCESS_TO_HEALTHCARE.DEI_HEALTHCARE."Delayed Healthcare Due to Cost - USA"
        """
        
        # if state:
        #     query4 += f" WHERE STATE = '{state}'"
            
        query4 += " GROUP BY YEAR"
        
        cursor.execute(query4)
        results4 = cursor.fetchall()
        column_names4 = [desc[0] for desc in cursor.description]
        df4 = pd.DataFrame(results4, columns=column_names4)
        results["delayed_healthcare_by_year"] = json.loads(df4.to_json(orient="records"))
        
        close_connection(cursor, conn)
        return json.dumps(results)
    
    except Exception as e:
        return f"Error executing healthcare access query: {str(e)}"

### Web tools

In [9]:
client = TavilyClient(TAVILY_API_KEY)
@tool
def web_search(query: str) -> str:
    """
    Searches the web for COVID-related information using Tavily api.
    
    Args:
        query (str): The search query, should be related to COVID-19, or healthcare in general.
    
    Returns:
        str: JSON string containing search results.
    """
    try:
        response = client.search(
        query=query
        )
        return response
    
    except Exception as e:
        print(f"Error in web search: {str(e)}")
        # Return empty results on error
        return json.dumps({"results": []})
        
    except Exception as e:
        return f"Error performing web search: {str(e)}"
    

# urls = ["https://www.nytimes.com/interactive/2021/us/covid-cases.html"]
@tool
def fetch_web_content(url: list) -> str:
    """
    Fetches content from a specific URL for detailed information.
    
    Args:
        url (list): List of URL's to fetch content from.
    
    Returns:
        str: The extracted content from the webpage.
    """
    try:
        response = client.extract(
            urls=url
        )
        return response["results"][0]["raw_content"]
    
    except Exception as e:
        print(f"Error in fetch web content: {str(e)}")
        # Return empty results on error
        return f"This is mock content about COVID-19 research and data analysis."
        
    except Exception as e:
        return f"Error fetching web content: {str(e)}"

### Main Run


In [10]:
model_id = "xai/grok-2-1212"
model = LiteLLMModel(model_id=model_id, api_key=os.getenv("XAI_API_KEY"))

In [11]:
def run_covid_analysis(state="Illinois"):
    """
    Main function to run the COVID-19 data analysis for a specified state.
    
    Args:
        state: The state to analyze (default: Illinois)
    
    Returns:
        Agent output containing the comprehensive report
    """
    # Initialize the model
    # model_id = "xai/grok-beta"
    # model = LiteLLMModel(model_id=model_id, api_key=os.getenv("XAI_API_KEY"))
    
    # Create the agent with all specialized tools
    agent = CodeAgent(
        tools=[
            query_covid_cases_by_year,
            query_vaccine_providers,
            query_healthcare_access,
            web_search,
            fetch_web_content
        ],
        model=model,
        max_steps=20,  # Increased for more complex workflow
        additional_authorized_imports=['pandas', 'json', 'requests', 'matplotlib', 'seaborn'],
        verbosity_level=2,
    )
    
    # Run the agent with a comprehensive prompt
    agent_output = agent.run(f"""
    You are a COVID-19 data analyst tasked with generating a comprehensive report on how the COVID-19 pandemic transformed healthcare in {state}.

    IMPORTANT GUIDELINES:
    - Only retry a tool call at most 3 times. If still not resolved, continue with existing data.
    - Process data one section at a time to minimize token usage.
    - For each section of the report, collect and analyze only the data needed for that section.
    - Clearly indicate when you're moving to a new section of the report.
    
    Follow this step-by-step approach:

    1. EXECUTIVE SUMMARY & INTRODUCTION:
       - Start by querying basic COVID data for context: query_covid_cases_by_year("{state}")
       - Search the web for an overview: web_search("{state} COVID-19 healthcare impact overview")
       - Write the Executive Summary and Introduction sections.

    2. PANDEMIC TIMELINE AND HEALTHCARE RESPONSE:
       - Analyze the COVID cases/deaths data already collected
       - Create this section based on the yearly trends.

    3. COMPARATIVE ANALYSIS:
       - Query healthcare access data: query_healthcare_access("{state}")
       - Focus on emergency department and physician visits data
       - Search for additional context: web_search("{state} pre-pandemic vs pandemic healthcare expenditure")
       - Complete this section using the collected data.

    4. SOCIAL DETERMINANTS AND COVID-19 IMPACT:
       - Analyze the delayed healthcare data from the healthcare_access query
       - Search for relevant information: web_search("{state} social determinants health COVID hotspots")
       - Complete this section with the findings.

    5. HEALTHCARE PROVIDER AVAILABILITY:
       - Query vaccination provider data: query_vaccine_providers("{state}")
       - Search for context: web_search("{state} healthcare provider availability COVID impact")
       - Complete this section with the collected data.

    6. LONG-TERM IMPLICATIONS:
       - Search for forecasting information: web_search("{state} long-term effects COVID healthcare access")
       - Complete this section based on search results.

    7. RECOMMENDATIONS AND CONCLUSION:
       - Draw on all previous analysis to formulate data-driven recommendations
       - Summarize key findings in the conclusion.

    The final report should follow this structure:
    
    # COVID-19 Impact on Healthcare in {state}: Comprehensive Analysis
    
    ## Executive Summary
    [Concise overview of key findings]
    
    ## Introduction
    [Context of COVID-19 in {state}, scope of the report]
    
    ## Pandemic Timeline and Healthcare Response
    [Use cases/deaths data to outline pandemic waves and response]
    
    ## Comparative Analysis: Pre-Pandemic vs. Pandemic Healthcare
    [Compare healthcare metrics before and during pandemic]
    
    ## Social Determinants and COVID-19 Impact
    [Identify vulnerable populations and impact patterns]
    
    ## Healthcare Provider Availability
    [Analyze vaccination provider distribution and access]
    
    ## Long-Term Implications
    [Forecast effects on healthcare access and policy implications]
    
    ## Recommendations
    [Data-driven recommendations for healthcare system resilience]
    
    ## Conclusion
    [Summarize key insights and future outlook]
    
    Make the report comprehensive (equivalent to 2-3 pages), evidence-based, and insightful for healthcare policymakers. Format with proper markdown, including headings, subheadings, bullet points, and emphasis where appropriate.
    """)
    
    return agent_output

In [12]:
output = run_covid_analysis()


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a COVID-19 data analyst tasked with generating a comprehensive report on how the COVID-19 pandemic      │
│ transformed healthcare in Illinois.                                                                             │
│                                                                                                                 │
│     IMPORTANT GUIDELINES:                                                                                       │
│     - Only retry a tool call at most 3 times. If still not resolved, continue with existing data.               │
│     - Process data one section at a time to minimize token usage.                                               │
│     - For each section of the report, collect and analyze only the data needed for that section.                │
│     - Clearly indicate when you're moving to a new section of the report.                                       │
│                                                                                                                 │
│     Follow this step-by-step approach:                                                                          │
│                                                                                                                 │
│     1. EXECUTIVE SUMMARY & INTRODUCTION:                                                                        │
│        - Start by querying basic COVID data for context: query_covid_cases_by_year("Illinois")                  │
│        - Search the web for an overview: web_search("Illinois COVID-19 healthcare impact overview")             │
│        - Write the Executive Summary and Introduction sections.                                                 │
│                                                                                                                 │
│     2. PANDEMIC TIMELINE AND HEALTHCARE RESPONSE:                                                               │
│        - Analyze the COVID cases/deaths data already collected                                                  │
│        - Create this section based on the yearly trends.                                                        │
│                                                                                                                 │
│     3. COMPARATIVE ANALYSIS:                                                                                    │
│        - Query healthcare access data: query_healthcare_access("Illinois")                                      │
│        - Focus on emergency department and physician visits data                                                │
│        - Search for additional context: web_search("Illinois pre-pandemic vs pandemic healthcare expenditure")  │
│        - Complete this section using the collected data.                                                        │
│                                                                                                                 │
│     4. SOCIAL DETERMINANTS AND COVID-19 IMPACT:                                                                 │
│        - Analyze the delayed healthcare data from the healthcare_access query                                   │
│        - Search for relevant information: web_search("Illinois social determinants health COVID hotspots")      │
│        - Complete this section with the findings.                                                               │
│                                                                                                                 │
│     5. HEALTHCARE PROVIDER AVAILABILITY:                                                                        │
│        - Query vaccination provider data: query_vaccin

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
### Task: Generate a comprehensive report on how the COVID-19 pandemic transformed healthcare in Illinois.         
                                                                                                                   
#### Step 1: EXECUTIVE SUMMARY & INTRODUCTION                                                                      
                                                                                                                   
**Thought:** To start the report, I need to gather basic data on COVID-19 cases in Illinois to set the context.    
This will help in crafting an informative Executive Summary and Introduction. I will use                           
`query_covid_cases_by_year` to fetch this data and `web_search` to get an overview of the impact on healthcare.    
                                                                                                                   
**Code:**                                                                                                          
```py                                                                                                              
covid_data_illinois = query_covid_cases_by_year(state="Illinois")                                                  
print("COVID-19 data for Illinois:", covid_data_illinois)                                                          
                                                                                                                   
overview = web_search(query="Illinois COVID-19 healthcare impact overview")                                        
print("Overview of Illinois COVID-19 healthcare impact:", overview)                                                
```                                                                                                                
<end_code>                                                                                                         
                                                                                                                   
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  covid_data_illinois = query_covid_cases_by_year(state="Illinois")                                                
  print("COVID-19 data for Illinois:", covid_data_illinois)                                                        
                                                                                                                   
  overview = web_search(query="Illinois COVID-19 healthcare impact overview")                                      
  print("Overview of Illinois COVID-19 healthcare impact:", overview)                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
COVID-19 data for Illinois: 
[{"YEAR":2020,"STATE":"Illinois","CASES":965840,"DEATHS":17979},{"YEAR":2021,"STATE":"Illinois","CASES":1188218,"DE
ATHS":13038},{"YEAR":2022,"STATE":"Illinois","CASES":1822782,"DEATHS":9661},{"YEAR":2023,"STATE":"Illinois","CASES"
:131091,"DEATHS":940}]
Overview of Illinois COVID-19 healthcare impact: {'query': 'Illinois COVID-19 healthcare impact overview', 
'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'PDF', 'url': 
'https://dph.illinois.gov/content/dam/soi/en/web/idph/publications/idph/topics-and-services/diseases-and-conditions
/respiratory-disease/covid19/after-action-report.pdf', 'content': 'State of Illinois - COVID-19 response overview 3
Lessons learned While Illinois demonstrated clear strengths in the COVID-19 response, as documented above, this 
review also identified a number of lessons learned to improve future responses to infectious disease public health 
emergencies. In some cases, the', 'score': 0.7570233, 'raw_content': None}, {'title': 'How COVID-19 changed 
Illinois: 5 impacts, 5 years later', 'url': 
'https://www.axios.com/local/chicago/2025/03/18/how-covid-19-changed-illinois', 'content': 'Residents line up for 
COVID-19 testing at Pritzker College Prep high school in the Hermosa neighborhood on Nov. 30, 2020. Photo: Scott 
Olson/Getty Images Public health . More than 40,000 Illinoisans have died from COVID-19 since 2020. The virus hit 
elderly people, those with preexisting conditions and the Black community especially hard.', 'score': 0.7500592, 
'raw_content': None}, {'title': 'Statewide Metrics - Illinois Department of Public Health', 'url': 
'https://dph.illinois.gov/covid19/data/statewide-metrics.html.', 'content': 'After decreases in cases, deaths, and 
hospitalizations related to COVID-19, IDPH is monitoring key indicators to identify early increases of virus 
transmission in Illinois, potentially signifying resurgence. These indicators are calculated for the entire state. 
IDPH will monitor if these indicators show an increase in the COVID-19 disease burden with a simultaneous decrease 
in', 'score': 0.62770075, 'raw_content': None}, {'title': 'Coronavirus Disease 2019 (COVID-19) - Illinois 
Department of Public Health', 'url': 'https://dph.illinois.gov/covid19.html]', 'content': "Coronavirus Disease 2019
(COVID-19) Your COVID-19 Toolkit Resources to keep yourself and others safe from COVID-19. Masks, Treatment, 
Vaccines, and Testing Resources near you. Data Last Updated ##/##/#### at 1 p.m. Data is updated on weekdays, 
Monday-Friday, either daily or weekly. Data is not updated on weekends or holidays. COVID-19 Resources COVID-19 
Outpatient Treatments What To Do if You Are Sick or Test Positive for COVID-19? COVID.GOV Your COVID-19 Resources 
Vaccination\xa0Data Search COVID-19 Guidance Illinois Data Visualizations COVID-19 Hospitalization Levels Data Last
Updated ##/##/#### This is a rapidly evolving situation and information will be updated as needed here and on the 
CDC website at https://www.cdc.gov/coronavirus/2019-ncov/index.html. For general questions about COVID-19 and 
Illinois' response and guidance, call 1-800-889-3931 or email\xa0dph.sick@illinois.gov RESOURCES", 'score': 
0.48308808, 'raw_content': None}, {'title': 'COVID-19 Data - Illinois Department of Public Health', 'url': 
'https://dph.illinois.gov/covid19/data.html.', 'content': 'Total Cases # Positivity 7-Day Rolling Average Case 
Positivity # Test Positivity # Total Tests Performed* # Deaths # Probable Deaths # Recovery Rate** # *Total 
molecular and antigen tests performed and reported electronically for testing of COVID-19 at IDPH, commercial or 
hospital laboratories. All numbers displayed are provisional and will change. Information regarding the', 'score': 
0.47504577, 'raw_content': None}], 'response_time': 2.4}

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 11 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
### Task: Generate a comprehensive report on how the COVID-19 pandemic transformed healthcare in Illinois.         
                                                                                                                   
#### Final Step: Compiling the Full Report                                                                         
                                                                                                                   
**Thought:** I have now completed all sections of the report on how the COVID-19 pandemic transformed healthcare in
Illinois. The report includes an Executive Summary, Introduction, Pandemic Timeline and Healthcare Response,       
Comparative Analysis, Social Determinants and COVID-19 Impact, Healthcare Provider Availability, Long-Term         
Implications, Recommendations, and Conclusion. I will now compile these sections into a single, comprehensive      
report and return it as the final answer.                                                                          
                                                                                                                   
**Code:**                                                                                                          
```py                                                                                                              
# Compiling the full report                                                                                        
full_report = f"""                                                                                                 
# COVID-19 Impact on Healthcare in Illinois: Comprehensive Analysis                                                
                                                                                                                   
{executive_summary}                                                                                                
                                                                                                                   
{introduction}                                                                                                     
                                                                                                                   
{timeline_and_response}                                                                                            
                                                                                                                   
{comparative_analysis}                                                                                             
                                                                                                                   
{social_determinants}                                                                                              
                                                                                                                   
{healthcare_provider_availability}                                                                                 
                                                                                                                   
{long_term_implications}                                                                                           
                                                                                                                   
{recommendations_and_conclusion}                                                                                   
"""                                                                                                                
                                                                                                                   
final_answer(full_report)                               

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Compiling the full report                                                                                      
  full_report = f"""                                                                                               
  # COVID-19 Impact on Healthcare in Illinois: Comprehensive Analysis                                              
                                                                                                                   
  {executive_summary}                                                                                              
                                                                                                                   
  {introduction}                                                                                                   
                                                                                                                   
  {timeline_and_response}                                                                                          
                                                                                                                   
  {comparative_analysis}                                                                                           
                                                                                                                   
  {social_determinants}                                                                                            
                                                                                                                   
  {healthcare_provider_availability}                                                                               
                                                                                                                   
  {long_term_implications}                                                                                         
                                                                                                                   
  {recommendations_and_conclusion}                                                                                 
  """                                                                                                              
                                                                                                                   
  final_answer(full_report)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
# COVID-19 Impact on Healthcare in Illinois: Comprehensive Analysis


## Executive Summary

The COVID-19 pandemic significantly transformed healthcare in Illinois, with notable impacts on case numbers, 
healthcare access, and social determinants of health. From 2020 to 2023, Illinois saw a total of 4,107,931 cases 
and 41,618 deaths. The healthcare system faced unprecedented challenges, leading to shifts in healthcare delivery, 
increased reliance on telehealth, and changes in healthcare expenditure. This report delves into these 
transformations, offering insights into the state's response, comparative analysis of healthcare metrics, and the 
long-term implications for healthcare policy and practice.



## Introduction

The COVID-19 pandemic, which began in early 2020, had a profound impact on Illinois. With a total of 4,107,931 
cases and 41,618 deaths recorded from 2020 to 2023, the state faced significant challenges in managing the health 
crisis. This report aims to provide a comprehensive analysis of how the pandemic transformed healthcare in 
Illinois. It will explore the timeline of the pandemic, the state's healthcare response, comparative analysis of 
healthcare metrics before and during the pandemic, the impact of social determinants on health outcomes, changes in
healthcare provider availability, and the long-term implications for healthcare access and policy. By understanding
these transformations, policymakers can better prepare for future health emergencies and improve healthcare 
resilience.



## Pandemic Timeline and Healthcare Response

The timeline of the COVID-19 pandemic in Illinois can be broken down by year:

- **2020**: The year the pandemic began, Illinois reported 965,840 cases and 17,979 deaths. The state's initial 
response included lockdowns, mask mandates, and the establishment of testing sites.

- **2021**: With 1,188,218 cases and 13,038 deaths, Illinois continued to manage the ongoing crisis. Vaccination 
efforts ramped up, and the state began to see a decrease in mortality rates due to increased immunity and better 
treatment protocols.

- **2022**: A significant increase in cases to 1,822,782 was observed, but deaths decreased to 9,661. This year saw
the emergence of new variants, leading to surges in cases, but the healthcare system adapted with increased testing
and vaccination drives.

- **2023**: By this year, cases dropped to 131,091 with 940 deaths, indicating a shift towards managing the virus 
as an endemic rather than a pandemic. The state continued to monitor and respond to outbreaks, focusing on booster 
shots and public health measures.

The healthcare response in Illinois involved:
- **Lockdowns and Restrictions**: Implemented to curb the spread of the virus.
- **Testing and Vaccination**: Expansion of testing sites and vaccination centers.
- **Healthcare System Adaptation**: Increased use of telehealth, reallocation of resources, and the development of 
new treatment protocols.



## Comparative Analysis: Pre-Pandemic vs. Pandemic Healthcare

### Emergency Department Visits
- **Pre-Pandemic (2018)**: 129,974 thousand visits
- **Pandemic (2020-2023)**: Data not available for these specific years, but we can infer from the trend that 
emergency department visits likely decreased due to fear of exposure and increased use of telehealth.

### Physician Visits
- **Pre-Pandemic (2018)**: 860,386 thousand visits
- **Pandemic (2020-2023)**: Similarly, data for these years is not available, but physician visits are expected to 
have decreased due to social distancing measures and the shift towards telehealth.

### Healthcare Expenditure
- **Pre-Pandemic**: Medicaid spending was less than $15 billion in 2010.
- **Pandemic**: Medicaid spending increased to more than $32 billion, indicating a significant rise in healthcare 
costs due to the increased demand and expanded coverage during the crisis.

The data suggests that while the number of in-person visits to healthcar

[Step 11: Duration 3.80 seconds| Input tokens: 117,630 | Output tokens: 3,773]

In [ ]:
from pprint import pprint
pprint(output)

('\n'
 '# COVID-19 Impact on Healthcare in Illinois: Comprehensive Analysis\n'
 '\n'
 '\n'
 '## Executive Summary\n'
 '\n'
 'The COVID-19 pandemic significantly transformed healthcare in Illinois, with '
 'notable impacts on case numbers, healthcare access, and social determinants '
 'of health. From 2020 to 2023, Illinois saw a total of 4,107,931 cases and '
 '41,618 deaths. The healthcare system faced unprecedented challenges, leading '
 'to shifts in healthcare delivery, increased reliance on telehealth, and '
 'changes in healthcare expenditure. This report delves into these '
 "transformations, offering insights into the state's response, comparative "
 'analysis of healthcare metrics, and the long-term implications for '
 'healthcare policy and practice.\n'
 '\n'
 '\n'
 '\n'
 '## Introduction\n'
 '\n'
 'The COVID-19 pandemic, which began in early 2020, had a profound impact on '
 'Illinois. With a total of 4,107,931 cases and 41,618 deaths recorded from '
 '2020 to 2023, the stat